In [113]:
import numpy as np
import pandas as pd
import tensorflow as tf
file = 'try_dayofweekonly.csv'
full_data = pd.read_csv(file)

output_feature = 'Base'
# subtract one from Base
full_data[output_feature] = full_data[output_feature] - 1
def split(df, ratio):
    msk = np.random.rand(len(df)) < ratio
    return df[msk], df[~msk]


train_data, test_data = split(full_data, 0.7)

train_data.head()


,ClusterCoordinateX,ClusterCoordinateY,DayOfWeekNum,ChunkIndex,Base
0,0,163,2,51,0
1,11,178,1,51,0
2,12,189,0,52,0
4,17,269,3,60,0
6,26,218,1,66,0


In [114]:
# input_features = ['x_coordinate', 'y_coordinate', 'dayofmonth', 'dayofweek', 'minchunk']
# input_features = train_data.columns[:5]
input_features = train_data.columns[:4]
get_data = tf.estimator.inputs.pandas_input_fn(train_data[input_features], train_data[output_feature], shuffle=True, num_epochs=2)



In [115]:
feature_cols = [tf.feature_column.numeric_column(k) for k in input_features]

model = tf.estimator.DNNClassifier([512, 256], feature_cols, n_classes=max(full_data[output_feature])+1, optimizer=lambda: tf.train.AdamOptimizer(learning_rate=0.0001))



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpkeqen08a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13d8c9198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [116]:
# start training
model.train(input_fn=get_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpkeqen08a/model.ckpt.
INFO:tensorflow:loss = 1452.5991, step = 1
INFO:tensorflow:global_step/sec: 346.737
INFO:tensorflow:loss = 69.83948, step = 101 (0.289 sec)
INFO:tensorflow:global_step/sec: 428.894
INFO:tensorflow:loss = 293.47775, step = 201 (0.233 sec)
INFO:tensorflow:global_step/sec: 362.758
INFO:tensorflow:loss = 29.543762, step = 301 (0.275 sec)
INFO:tensorflow:global_step/sec: 438.049
INFO:tensorflow:loss = 34.346786, step = 401 (0.228 sec)
INFO:tensorflow:global_step/sec: 431.392
INFO:tensorflow:loss = 51.19761, step = 501 (0.232 sec)
INFO:tensorflow:global_step/sec: 396.032
INFO:tensorflow:loss = 55.860046, step = 601 (0.252 sec)
INFO:tensorflow

In [117]:
# evaluate metrics
get_test_data = tf.estimator.inputs.pandas_input_fn(test_data[input_features], test_data[output_feature], shuffle=True)
model.evaluate(input_fn=get_test_data)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T07:41:08Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpkeqen08a/model.ckpt-5098
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-06-07:41:10
INFO:tensorflow:Saving dict for global step 5098: accuracy = 0.8244443, average_loss = 0.8125759, global_step = 5098, loss = 103.96371
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5098: /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpkeqen08a/model.ckpt-5098


{'accuracy': 0.8244443,
 'average_loss': 0.8125759,
 'loss': 103.96371,
 'global_step': 5098}

In [80]:
# predict
prediction_data = test_data[5:8]
print(prediction_data.head())

get_prediction_data = tf.estimator.inputs.pandas_input_fn(prediction_data[input_features], prediction_data[output_feature], shuffle=False, num_epochs=1)
list(model.predict(input_fn=get_prediction_data))

    ClusterCoordinateX  ClusterCoordinateY  DayOfWeekNum  MonthDayNum  \
15                  36                 165             4           19   
16                  38                 132             3           10   
22                  39                 221             4           22   

    ChunkIndex  Base  
15          58     0  
16          72     0  
22          56     0  
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpzfmk60oz/model.ckpt-56010
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([  5.200681 ,   4.121735 ,   2.5672936,   0.5282359,  -0.6152709,
          -1.6932842,  -3.6285505,  -3.2394896,  -6.5522223,  -6.7849503,
          -5.815725 ,  -4.9695363,  -6.8484   ,  -7.8592367,  -7.9865947,
          -7.0638967, -10.532853 ,  -9.180073 , -11.860085 , -11.416455 ,
         -10.74991  , -11.420926 , -10.722576 , -10.52589  , -14.5951605,
         -13.328194 , -16.523994 , -14.433364 , -15.433117 , -15.02038  ,
         -18.617886 , -18.026608 , -15.98516  , -18.927778 , -20.464872 ,
         -20.292479 , -18.602755 , -19.558033 , -21.163078 , -19.208973 ,
         -21.783129 , -26.943079 , -20.222704 , -26.182396 , -20.461061 ,
         -19.480263 , -28.558489 , -23.645895 , -24.069824 , -29.369898 ,
         -33.467335 , -31.330029 , -28.927286 , -29.368849 , -30.55372  ,
         -27.231928 , -20.918932 , -28.307497 , -29.598791 , -27.188847 ,
         -21.73226  , -28.781767 , -21.39126  , -27.57989  , -29.505196 ,
         -29.245588 , -26.86